In [19]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import functions as fun
import statsmodels.api as sm
import seaborn as sns
from linearmodels.iv import IV2SLS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data['Model_year'] = df['Model'] + '_' + df['Year'].astype(str)

data.head(20)

/var/folders/ys/szn3cjzn7zg_mdp21slh2_hr0000gn/T/ipykernel_68452/2282954133.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Model_year'] = df['Model'] + '_' + df['Year'].astype(str)


,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,Model_year
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0,U5_2021
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0,U5_2022
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0,U5_2023
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0,U6_2023
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,e-tron_2019
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0,e-tron_2020
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0,e-tron_2021
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0,e-tron_2022
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0,e-tron_2023
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0,e-tron GT_2021


In [21]:
# Copy the dataframe
df2 = data.copy()

In [22]:
#Scale for better intepretation

df2['Price'] = df2['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
df2['HP'] = df2['HP']/10           #(Change in ms(%) for change in HP in 10)
df2['Range'] = df2['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [23]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CN').astype(int)

In [24]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# OLS

In [25]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'Price', 'HP', 'Chargetime']]
dummies = df2[['China']]
X = pd.concat([x, dummies], axis=1)
print(X.shape)

(334, 6)


In [26]:
OLS_model = sm.OLS(y, X)
OLS_result = OLS_model.fit(cov_type='HC3')
# Print summary of the regression results
print(OLS_result.summary())

                            OLS Regression Results                            
Dep. Variable:       log_market_share   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     21.01
Date:                Thu, 09 May 2024   Prob (F-statistic):           3.18e-18
Time:                        11:45:41   Log-Likelihood:                -716.37
No. Observations:                 334   AIC:                             1445.
Df Residuals:                     328   BIC:                             1468.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.4240      0.700     -9.171      0.0

# BLP instrument (sum)

In [27]:
fun.BLP(df2, 'HP')
fun.BLP(df2, 'Chargetime')
fun.BLP(df2, 'Range')

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,Model_year,China,log_market_share,HP_sum,Chargetime_sum,Range_sum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CN,257,1.0,U5_2021,1,-4.568514,1358.3,1861,2151.3
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CN,183,1.0,U5_2022,1,-5.119955,2033.9,2664,3218.0
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CN,177,1.0,U5_2023,1,-5.856883,2640.3,3241,4177.4
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CN,3,1.0,U6_2023,1,-9.934421,2639.0,3241,4176.9
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0,e-tron_2019,0,-3.203259,390.1,570,580.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,B,SE,4,1.0,EX30_2023,0,-9.646739,2633.6,3247,4169.9
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,C,SE,363,1.0,XC40_2021,0,-4.223188,1338.2,1867,2145.6
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,C,SE,1020,1.0,XC40_2022,0,-3.401883,2013.8,2670,3212.3
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,C,SE,1965,1.0,XC40_2023,0,-3.449786,2620.2,3247,4171.7


In [28]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['Range_sum', 'HP_sum', 'Chargetime_sum']] 

In [29]:
BLP_model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

In [30]:
BLP_model.first_stage

,Price
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
const,-1.3281
,(-0.1735)
Range,0.1875


In [31]:
BLP_model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -5.7877
Estimator:                    IV-2SLS   Adj. R-squared:                -5.8912
No. Observations:                 334   F-statistic:                    28.868
Date:                Thu, May 09 2024   P-value (F-stat)                0.0000
Time:                        11:45:41   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -2.8041     2.3134    -1.2121     0.2255     -7.3383      1.7300
Range          0.1995     0.0588     3.3923     0.0007      0.0842      0.3148
HP             0.3270     0.0955     3.4228     0.0006      0.1398      0.5143
Chargetime    -0.0848     0.0305    -2.7760     0.0055     -0.1447     -0.0249
China         -5.4735     1.1202    -4.8862     0.0000     -7.6690     -3.2779
Price         -0.3222     0.0761    -4.2313     0.0000     -0.4714     -0.1729
==============================================================================

Endogenous: Price
Instruments: Range_sum, HP_sum, Chargetime_sum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde instrument (Local Diff)

In [32]:
fun.create_instrument_localsum(df2, 'HP', 0.5)
fun.create_instrument_localsum(df2, 'Chargetime', 0.5)
fun.create_instrument_localsum(df2, 'Range', 0.5)

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:179: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '740.5000000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, instrument + '_instrument_localsum'] = sum_except_current
/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:179: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1499.9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, instrument + '_instrument_localsum'] = sum_except_current


,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,const,Model_year,China,log_market_share,HP_sum,Chargetime_sum,Range_sum,HP_instrument_localsum,Chargetime_instrument_localsum,Range_instrument_localsum
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,...,1.0,U5_2021,1,-4.568514,1358.3,1861,2151.3,740.5,1317,1499.9
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,...,1.0,U5_2022,1,-5.119955,2033.9,2664,3218.0,1060.2,1796,2153.5
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,...,1.0,U5_2023,1,-5.856883,2640.3,3241,4177.4,1501.0,2099,2792.3
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,...,1.0,U6_2023,1,-9.934421,2639.0,3241,4176.9,1456.2,2099,2746.8
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,...,1.0,e-tron_2019,0,-3.203259,390.1,570,580.9,310.5,533,470.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,47.5,36.824500,26.8,28,SUV,...,1.0,EX30_2023,0,-9.646739,2633.6,3247,4169.9,1777.6,1813,2189.4
1163,190,2021,0.014652,Volvo,XC40,45.7,46.206060,40.2,28,SUV,...,1.0,XC40_2021,0,-4.223188,1338.2,1867,2145.6,1026.5,1138,1169.9
1164,190,2022,0.033310,Volvo,XC40,45.7,41.626340,40.2,28,SUV,...,1.0,XC40_2022,0,-3.401883,2013.8,2670,3212.3,1621.7,1483,1702.8
1165,190,2023,0.031752,Volvo,XC40,45.7,43.926660,40.2,28,SUV,...,1.0,XC40_2023,0,-3.449786,2620.2,3247,4171.7,1978.6,1813,2206.7


In [33]:
z=df2[['Range_instrument_localsum', 'HP_instrument_localsum', 'Chargetime_instrument_localsum']] 
GH_model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
GH_model.first_stage

,Price
R-squared,0.6105
Partial R-squared,0.0919
Shea's R-squared,0.0919
Partial F-statistic,22.829
P-value (Partial F-stat),4.383e-05
Partial F-stat Distn,chi2(3)
================================,===========
const,1.7422
,(0.2392)
Range,0.4165


In [34]:
GH_model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -1.1894
Estimator:                    IV-2SLS   Adj. R-squared:                -1.2228
No. Observations:                 334   F-statistic:                    73.809
Date:                Thu, May 09 2024   P-value (F-stat)                0.0000
Time:                        11:45:42   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -4.6767     1.4901    -3.1386     0.0017     -7.5972     -1.7562
Range          0.1395     0.0231     6.0487     0.0000      0.0943      0.1847
HP             0.1587     0.0585     2.7127     0.0067      0.0440      0.2734
Chargetime    -0.0478     0.0201    -2.3749     0.0176     -0.0872     -0.0083
China         -3.7943     0.6345    -5.9798     0.0000     -5.0380     -2.5507
Price         -0.1722     0.0448    -3.8463     0.0001     -0.2599     -0.0844
==============================================================================

Endogenous: Price
Instruments: Range_instrument_localsum, HP_instrument_localsum, Chargetime_instrument_localsum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Logit

In [35]:
GH_params = GH_model.params
GH_params = np.array(GH_params)
GH_params = np.array([GH_params[0], GH_params[-1], *GH_params[1:-1]])
GH_params

array([-4.67668433, -0.17218025,  0.13951338,  0.15870948, -0.04775668,
       -3.79431677])

In [36]:
logit_df = df.copy()
#Scale for better intepretation
logit_df['Price'] = logit_df['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
logit_df['HP'] = logit_df['HP']/10           #(Change in ms(%) for change in HP in 10)
logit_df['Range'] = logit_df['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)
logit_df = fun.straf_0ms(logit_df)

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:187: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Price'][i] = 10_000_000
/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:187: SettingWithCopyWarning: 
A value is

In [37]:
logit_df['China'] = (logit_df['Country'] == 'CN').astype(int)
x = logit_df[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = logit_df[['China']]
X = pd.concat([x, dummies], axis=1)

alpha = GH_params[1]
alpha = np.array(alpha)
beta = GH_params
beta = np.array(beta)
p_j = logit_df['Price']
p_j = np.array(p_j)
columns = X.columns[1:]
X = np.array(X)

In [38]:
logit_df['logit_market_share'] = fun.ccp(alpha, beta, X, p_j)
logit_df2 = logit_df[logit_df['Market share'] != 0]
ccp = logit_df2['logit_market_share']
ccp = np.array(ccp)

AttributeError: 'numpy.ndarray' object has no attribute 'groupby'

In [ ]:
probability_ratio = fun.probability_ratio(ccp, logit_df2['Model'])
probability_ratio

NameError: name 'ccp' is not defined

In [ ]:
marginal_effects = fun.marginal_effects(ccp, logit_df2['Model'], columns[1:], beta)
marginal_effects

NameError: name 'ccp' is not defined

In [ ]:
elasticity = fun.elasticity(ccp, logit_df2['Model'], columns[1:], beta, X)
elasticity

NameError: name 'ccp' is not defined

In [ ]:
cross_elasticity = fun.cross_elasticity(ccp, beta, X, np.array(logit_df2['Model']))
cross_elasticity

NameError: name 'ccp' is not defined